### Restart and Run All Cells

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2022
quarter = 3
today = date.today()
today_str = today.strftime("%Y-%m-%d")
today_str

'2022-11-10'

In [2]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit percent'.split()

In [3]:
format_dict = {
                'q_amt':'{:,}','q_amt_c':'{:,}','q_amt_p':'{:,}','inc_profit':'{:,}',
                'yoy_gain':'{:,}','acc_gain':'{:,}',   
                'latest_amt':'{:,}','previous_amt':'{:,}','inc_amt':'{:,}',
                'q_eps':'{:.4f}','y_eps':'{:.4f}','aq_eps':'{:.4f}','ay_eps':'{:.4f}',
                'percent':'{:.2f}%','inc_pct':'{:.2f}%'
              }

In [4]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC
"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 3) 
OR (year = 2022-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC



In [5]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.head()

,name,year,quarter,q_amt,Counter
1,ADVANC,8087,10,25511368,4
6,AIT,8087,10,582222,4
9,ANAN,8087,10,-560806,4
10,AOT,8087,10,-14912794,4
11,AP,8087,10,5716603,4


In [6]:
sql = """
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2022 AND quarter <= 3-1) 
OR (year = 2022-1 AND quarter >= 3) 
ORDER BY year DESC, quarter DESC


In [7]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.head().style.format(format_dict)

,name,year,quarter,q_amt,Counter
0,ACE,8086,10,"1,485,354",4
1,ADVANC,8086,10,"25,853,429",4
2,AEONTS,8086,10,"3,743,905",4
3,AH,8086,10,"1,175,297",4
4,AIE,8086,10,"402,015",4


In [8]:
dfp.name.unique().shape

(233,)

In [9]:
sql = """
SELECT *
FROM stocks
"""
stocks = pd.read_sql(sql, conlt)
stocks.shape

(233, 15)

In [10]:
sql = """
SELECT *
FROM tickers
"""
tickers = pd.read_sql(sql, conlt)
tickers.shape

(401, 9)

In [11]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["percent"] = round(dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100, 2)
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_percent = dfm[cols]
df_percent.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2022,Q3,"25,511,368","25,853,429","-342,061",-1.32%
1,AIT,2022,Q3,"582,222","556,639","25,583",4.60%
2,ANAN,2022,Q3,"-560,806","-796,635","235,829",29.60%
3,AOT,2022,Q3,"-14,912,794","-16,784,073","1,871,279",11.15%
4,AP,2022,Q3,"5,716,603","5,329,148","387,455",7.27%


In [12]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'
"""
sql = sql % (year, quarter)
print(sql)

rp = conlt.execute(sql)
rp.rowcount


DELETE FROM qt_profits 
WHERE year = 2022 AND quarter = 'Q3'



104

In [13]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
tickers.head()

,name,id
0,2S,727
1,A,1
2,AAV,732
3,ACE,698
4,ADVANC,6


In [14]:
df_ins = pd.merge(df_percent, tickers, on="name", how="inner")
df_ins.head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
0,ADVANC,2022,Q3,"25,511,368","25,853,429","-342,061",-1.32%,6
1,AIT,2022,Q3,"582,222","556,639","25,583",4.60%,11
2,ANAN,2022,Q3,"-560,806","-796,635","235,829",29.60%,23
3,AOT,2022,Q3,"-14,912,794","-16,784,073","1,871,279",11.15%,24
4,AP,2022,Q3,"5,716,603","5,329,148","387,455",7.27%,25


In [15]:
rcds = df_ins.values.tolist()
len(rcds)

131

In [16]:
#for rcd in rcds:
#    print(rcd)

In [17]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, \
inc_amt, inc_pct, ticker_id) \
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [18]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [19]:
criteria_1 = df_ins.q_amt_c > 440_000
df_ins.loc[criteria_1, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2022,Q3,"25,511,368","25,853,429","-342,061",-1.32%
1,AIT,2022,Q3,"582,222","556,639","25,583",4.60%
4,AP,2022,Q3,"5,716,603","5,329,148","387,455",7.27%
5,ASIAN,2022,Q3,"1,020,743","1,048,954","-28,211",-2.69%
6,ASP,2022,Q3,"489,367","578,771","-89,404",-15.45%


In [20]:
criteria_2 = df_ins.q_amt_p > 400_000
df_ins.loc[criteria_2, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
0,ADVANC,2022,Q3,"25,511,368","25,853,429","-342,061",-1.32%
1,AIT,2022,Q3,"582,222","556,639","25,583",4.60%
4,AP,2022,Q3,"5,716,603","5,329,148","387,455",7.27%
5,ASIAN,2022,Q3,"1,020,743","1,048,954","-28,211",-2.69%
6,ASP,2022,Q3,"489,367","578,771","-89,404",-15.45%


In [21]:
criteria_3 = df_ins.percent > 10.00
df_ins.loc[criteria_3, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
2,ANAN,2022,Q3,"-560,806","-796,635","235,829",29.60%
3,AOT,2022,Q3,"-14,912,794","-16,784,073","1,871,279",11.15%
8,AWC,2022,Q3,"3,414,980","3,075,541","339,439",11.04%
10,BANPU,2022,Q3,"44,283,431","30,044,483","14,238,948",47.39%
16,BE8,2022,Q3,"110,853","96,714","14,139",14.62%


In [22]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria, cols].head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent
8,AWC,2022,Q3,"3,414,980","3,075,541","339,439",11.04%
10,BANPU,2022,Q3,"44,283,431","30,044,483","14,238,948",47.39%
18,BH,2022,Q3,"4,004,426","2,799,073","1,205,353",43.06%
19,BJC,2022,Q3,"4,772,996","4,207,791","565,205",13.43%
20,BPP,2022,Q3,"6,289,501","4,570,808","1,718,693",37.60%


In [23]:
df_ins[df_ins_criteria].sort_values(by=["percent"], ascending=[False]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
110,TFG,2022,Q3,"3,733,094","1,543,647","2,189,447",141.84%,513
23,CPNREIT,2022,Q3,"3,142,324","1,624,073","1,518,251",93.48%,647
35,GFPT,2022,Q3,"1,646,990","876,315","770,675",87.94%,187
64,M,2022,Q3,"1,517,262","851,758","665,504",78.13%,279
87,SABUY,2022,Q3,"1,041,005","596,382","444,623",74.55%,721


In [24]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
8,AWC,2022,Q3,"3,414,980","3,075,541","339,439",11.04%,699
10,BANPU,2022,Q3,"44,283,431","30,044,483","14,238,948",47.39%,47
18,BH,2022,Q3,"4,004,426","2,799,073","1,205,353",43.06%,61
19,BJC,2022,Q3,"4,772,996","4,207,791","565,205",13.43%,65
20,BPP,2022,Q3,"6,289,501","4,570,808","1,718,693",37.60%,74


In [25]:
df_ins[df_ins_criteria].sort_values(by=["name"], ascending=[True]).head().style.format(format_dict)

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,percent,id
8,AWC,2022,Q3,"3,414,980","3,075,541","339,439",11.04%,699
10,BANPU,2022,Q3,"44,283,431","30,044,483","14,238,948",47.39%,47
18,BH,2022,Q3,"4,004,426","2,799,073","1,205,353",43.06%,61
19,BJC,2022,Q3,"4,772,996","4,207,791","565,205",13.43%,65
20,BPP,2022,Q3,"6,289,501","4,570,808","1,718,693",37.60%,74


In [26]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Group of 4 Quarters'
FROM qt_profits
GROUP BY name
HAVING COUNT(*) = 1
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Group of 4 Quarters
0,BE8,1
1,III,1
2,SABUY,1


In [27]:
name = 'OR'
sql = """
SELECT *
FROM epss
WHERE name = '%s'
ORDER BY year DESC, quarter DESC
"""
sql = sql % name
print(sql)

epss = pd.read_sql(sql, conlt)
epss


SELECT *
FROM epss
WHERE name = 'OR'
ORDER BY year DESC, quarter DESC



,id,name,year,quarter,q_amt,y_amt,aq_amt,ay_amt,q_eps,y_eps,aq_eps,ay_eps,ticker_id,publish_date
0,21952,OR,2022,3,700989,1892478,11113908,9120618,0.06,0.16,0.93,0.79,715,2022-11-08
1,21822,OR,2022,2,6567811,3224942,10412919,7228140,0.55,0.27,0.87,0.64,715,2022-08-11
2,21463,OR,2022,1,3845107,4003198,3845107,4003198,0.32,0.38,0.32,0.38,715,2022-05-10
3,21135,OR,2021,4,2353412,2922577,11474030,8791070,0.20,0.33,0.99,0.98,715,2022-02-15
4,20753,OR,2021,3,1892478,3450043,9120618,5868493,0.16,0.38,0.79,0.65,715,2021-11-09
5,20266,OR,2021,2,3224942,520104,7228140,2418450,0.27,0.06,0.64,0.27,715,2021-08-10
6,19788,OR,2021,1,4003198,1898346,4003198,1898346,0.38,0.21,0.38,0.21,715,2021-05-11
7,19258,OR,2020,4,2922576,1948351,8791070,10895805,0.33,0.22,0.98,1.21,715,2021-02-17
8,19257,OR,2020,3,3450043,2593929,5868494,8947454,0.38,0.29,0.65,0.99,715,2021-02-17


In [28]:
sql = """
SELECT name, COUNT(*) AS 'Nbr. of Quarters'
FROM epss
GROUP BY name
HAVING COUNT(*) < 5
ORDER BY COUNT(*) DESC, name
"""
print(sql)

tmp = pd.read_sql(sql, conlt)
tmp


SELECT name, COUNT(*) AS 'Nbr. of Quarters'
FROM epss
GROUP BY name
HAVING COUNT(*) < 5
ORDER BY COUNT(*) DESC, name



,name,Nbr. of Quarters
0,ONEE,4
